In [1]:
import os
#Generate an HDFS Delegation token
#%env HADOOP_TOKEN_FILE_LOCATION=/mnt/data/$ON-DEMAND-SPARK/hdfsdt.token
#Run the following commands
!/mnt/code/scripts/my_hdfs.sh fetchdt --renewer null /mnt/data/$DOMINO_PROJECT_NAME/hdfsdt.token
%env HADOOP_TOKEN_FILE_LOCATION=/mnt/data/$DOMINO_PROJECT_NAME/hdfsdt.token
hdfs_endpoint=os.environ['HDFS_ENDPOINT']
os.environ['HADOOP_TOKEN_FILE_LOCATION']

2021-08-14 23:54:45,120 INFO hdfs.DFSClient: Created token for dominospark: HDFS_DELEGATION_TOKEN owner=dominospark@KDCDOMINO.COM, renewer=null, realUser=, issueDate=1628985285106, maxDate=1629590085106, sequenceNumber=64, masterKeyId=3 on 10.0.123.114:8020
env: HADOOP_TOKEN_FILE_LOCATION=/mnt/data/$DOMINO_PROJECT_NAME/hdfsdt.token


'/mnt/data/$DOMINO_PROJECT_NAME/hdfsdt.token'

In [2]:
%env PYSPARK_PYTHON /usr/bin/python3
%env PYSPARK_DRIVER_PYTHON /usr/bin/python3
#%env SPARK_DIST_CLASSPATH  /etc/hadoop/conf:/opt/hadoop/share/hadoop/common/lib/*:/opt/hadoop/share/hadoop/common/*:/usr/lib/hadoop-lzo/lib/hadoop-lzo-0.4.19.jar:/usr/lib/hadoop-lzo/lib/hadoop-lzo.jar:/usr/lib/hadoop-lzo/lib/native:/usr/share/aws/aws-java-sdk/aws-java-sdk-bundle-1.11.977.jar:/usr/share/aws/aws-java-sdk/LICENSE.txt:/usr/share/aws/aws-java-sdk/NOTICE.txt:/usr/share/aws/aws-java-sdk/README.md:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/annotations-16.0.2.jar:/usr/share/aws/emr/emrfs/lib/aopalliance-1.0.jar:/usr/share/aws/emr/emrfs/lib/aws-glue-sdk-1.12.0.jar:/usr/share/aws/emr/emrfs/lib/bcprov-ext-jdk15on-1.66.jar:/usr/share/aws/emr/emrfs/lib/emrfs-hadoop-assembly-2.46.0.jar:/usr/share/aws/emr/emrfs/lib/fluent-hc-4.5.6.jar:/usr/share/aws/emr/emrfs/lib/ion-java-1.0.2.jar:/usr/share/aws/emr/emrfs/lib/javax.inject-1.jar:/usr/share/aws/emr/emrfs/lib/jcl-over-slf4j-1.7.21.jar:/usr/share/aws/emr/emrfs/lib/jmespath-java-1.11.852.jar:/usr/share/aws/emr/emrfs/lib/lombok-1.18.4.jar:/usr/share/aws/emr/emrfs/lib/mockito-core-1.10.19.jar:/usr/share/aws/emr/emrfs/lib/objenesis-2.1.jar:/usr/share/aws/emr/emrfs/lib/secret-agent-interface-1.4.0.jar:/usr/share/aws/emr/emrfs/lib/slf4j-api-1.7.21.jar:/usr/share/aws/emr/emrfs/auxlib/*:/usr/share/aws/emr/ddb/lib/emr-ddb-hadoop.jar:/usr/share/aws/emr/goodies/lib/emr-hadoop-goodies.jar:/usr/share/aws/emr/kinesis/lib/emr-kinesis-hadoop.jar:/usr/share/aws/emr/cloudwatch-sink/lib/cloudwatch-sink-2.1.0.jar:/usr/share/aws/emr/cloudwatch-sink/lib/cloudwatch-sink.jar:/opt/hadoop/share/hadoop/tools/lib/*:/usr/lib/hadoop-lzo/lib/*
#%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64
hdfs_endpoint=os.environ['HDFS_ENDPOINT']
hdfs_endpoint
hdfs_dt_token_path=os.environ['HADOOP_TOKEN_FILE_LOCATION']
os

env: PYSPARK_PYTHON=/usr/bin/python3
env: PYSPARK_DRIVER_PYTHON=/usr/bin/python3


<module 'os' from '/opt/conda/lib/python3.7/os.py'>

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random

In [4]:
#On DEMAND SPARK WITH DT
%env PYSPARK_PYTHON /opt/conda/bin/python3
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python3

sparkSession = SparkSession.builder.appName("Calculate Pi using OnDemand Spark") \
.config("spark.dynamicAllocation.enabled", "false") \
.config("fs.default.name", hdfs_endpoint) \
.config("spark.driver.extraClassPath", "/opt/hadoop/etc/hadoop:/usr/lib/hadoop-lzo/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/hdfs/*") \
.config("spark.executorEnv.HADOOP_TOKEN_FILE_LOCATION",hdfs_dt_token_path) \
.config("spark.driverEnv.HADOOP_TOKEN_FILE_LOCATION",hdfs_dt_token_path) \
.getOrCreate()
sc=sparkSession.sparkContext
#.config("spark.executor.extraClassPath", "/opt/hadoop/etc/hadoop:/usr/lib/hadoop-lzo/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/hdfs/*") \

env: PYSPARK_PYTHON=/opt/conda/bin/python3
env: PYSPARK_DRIVER_PYTHON=/opt/conda/bin/python3


Exception: Java gateway process exited before sending its port number

In [ ]:
'''
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python

sparkSession = SparkSession.builder.appName("Calculate Pi using Local Spark") \
.master("local[*]") \
.config("spark.dynamicAllocation.enabled", "false") \
.config("fs.default.name", hdfs_endpoint) \
.config("spark.driver.extraClassPath", "/opt/hadoop/etc/hadoop:/usr/lib/hadoop-lzo/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/hdfs/*") \
.getOrCreate()
sc=sparkSession.sparkContext
#.config("spark.executor.extraClassPath", "/opt/hadoop/etc/hadoop:/usr/lib/hadoop-lzo/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/hdfs/*") \
'''

In [8]:
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1


In [9]:


columns = StructType([ StructField("name", StringType(), True),
                      StructField("value", DoubleType(), True)
                    ])

count = sc.parallelize(range(0, 1000),1) \
             .filter(inside).count()
data = [("Pi",4.0 * count/1000)]

df = sparkSession.createDataFrame(data=data, schema=columns)

df.show()

NameError: name 'sc' is not defined

In [6]:
#Let us write to a dataset
ds_path = '/user/dominospark/my-pi'
!/mnt/code/scripts/my_hdfs.sh dfs -rmr '/user/dominospark/my-pi*'
df.write.csv(ds_path)
#Read it back
sparkSession.read.csv(ds_path).show()


rmr: DEPRECATED: Please use '-rm -r' instead.
Deleted /user/dominospark/my-pi
+---+-----+
|_c0|  _c1|
+---+-----+
| Pi|3.188|
+---+-----+



In [7]:
!/mnt/code/scripts/my_hdfs.sh dfs -rmr '/user/dominospark/small-data-100/'
hdfs_src_path = '/user/dominospark/largedata/'
hdfs_dest_path =  '/user/dominospark/small-data-100/'
local_dest_path = 'file:///mnt/data/ON-DEMAND-SPARK/small-data-100'
!rm -rf /mnt/data/ON-DEMAND-SPARK/small-data-100
filter_criteria = 100
sparkSession = SparkSession.builder.appName("Generate Data") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("fs.default.name", hdfs_endpoint) \
    .getOrCreate()

    
columns = StructType([ StructField("id", IntegerType(), True), \
                       StructField("v1", IntegerType(), True),\
                       StructField("v2", IntegerType(), True),\
                       StructField("v3", IntegerType(), True) ])

df_load = sparkSession.read.csv(hdfs_src_path,columns)
df_load_filtered = df_load.where(df_load.id < filter_criteria)
df_load_filtered.write.csv(hdfs_dest_path)
df_load_filtered.write.csv(local_dest_path)


rmr: DEPRECATED: Please use '-rm -r' instead.
Deleted /user/dominospark/small-data-100


In [8]:
df_load_read = sparkSession.read.csv(hdfs_dest_path,columns)
df_load_read.show()

+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|  0| 63| 94| 50|
|  1| 25| 26| 73|
|  2| 84| 84| 84|
|  3| 47| 19|  8|
|  4| 24| 31|  6|
|  5| 75| 17| 11|
|  6| 49| 38| 57|
|  7| 56| 31| 90|
|  8|100|  3|  3|
|  9| 43| 72| 34|
| 10| 18| 64| 57|
| 11| 63| 75| 80|
| 12| 82| 85| 28|
| 13| 31|  8| 42|
| 14| 20| 80|  3|
| 15| 27| 91| 86|
| 16| 55| 70| 42|
| 17| 69|  3|  5|
| 18| 65| 28| 28|
| 19| 57|  8| 69|
+---+---+---+---+
only showing top 20 rows



In [10]:
df_load_read = sparkSession.read.csv(local_dest_path,columns)
df_load_read.show()

+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|  0| 63| 94| 50|
|  1| 25| 26| 73|
|  2| 84| 84| 84|
|  3| 47| 19|  8|
|  4| 24| 31|  6|
|  5| 75| 17| 11|
|  6| 49| 38| 57|
|  7| 56| 31| 90|
|  8|100|  3|  3|
|  9| 43| 72| 34|
| 10| 18| 64| 57|
| 11| 63| 75| 80|
| 12| 82| 85| 28|
| 13| 31|  8| 42|
| 14| 20| 80|  3|
| 15| 27| 91| 86|
| 16| 55| 70| 42|
| 17| 69|  3|  5|
| 18| 65| 28| 28|
| 19| 57|  8| 69|
+---+---+---+---+
only showing top 20 rows



In [11]:
sparkSession.stop()

In [12]:
!/mnt/code/scripts/my_hdfs.sh dfs -ls

Found 11 items
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 18:53 .sparkStaging
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 20:45 example
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 21:45 large-data
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 18:55 largedata
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 21:57 ld-10
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 19:27 my-pi
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 21:07 mypi
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 21:58 sd-5
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 19:27 small-data-100
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 19:22 smalldata-10
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 18:53 smalldata-1000


'/opt/spark'